STEP 1 — Install & Import Libraries

In [1]:
# Install required package (run once)
!pip install tensorflow tensorflow-hub matplotlib boto3

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 258.0 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tensorflow-hub]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


2025-12-23 14:38:17.380414: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 14:38:24.707271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-23 14:38:28.675514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-23 14:38:28.716690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-23 14:38:34.381953: I tensorflow/core/platform/cpu_feature_gua

STEP 2 — Configuration & Paths

In [2]:
print("TensorFlow version:", tf.__version__)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 300

# Local directory (downloaded from S3 previously)
train_dir = "synthetic_data/Training Data"

print("Training directory:", train_dir)
print("Epochs:", EPOCHS)


TensorFlow version: 2.16.2
Training directory: synthetic_data/Training Data
Epochs: 300


STEP 3 — Load Data (20% Validation Split, NO Augmentation)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

print("Class indices:", train_generator.class_indices)


STEP 4 — Load EfficientNet-Lite Backbone (Frozen)

In [ ]:
# TF-Hub EfficientNet-Lite0 (ImageNet pretrained)
EFFICIENTNET_LITE_URL = (
    "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"
)

# Load feature extractor
base_model = hub.KerasLayer(
    EFFICIENTNET_LITE_URL,
    input_shape=(224, 224, 3),
    trainable=False   # IMPORTANT: frozen backbone
)

print("EfficientNet-Lite backbone loaded and frozen")


STEP 5 — Build Classification Head

In [ ]:
from tensorflow.keras.layers import Input

# Create input tensor
inputs = Input(shape=(224, 224, 3))

# Pass inputs through EfficientNet-Lite backbone
x = base_model(inputs)   # ✅ THIS is the key fix

# Add classification head
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

# Create final model
model = Model(inputs=inputs, outputs=outputs)

# Print model summary
model.summary()


STEP 6 — Compile Model

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[BinaryAccuracy(name="accuracy", threshold=0.5)]
)

print("Model compiled successfully")


STEP 7 — Callbacks

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=5,
    min_lr=1e-7,
    verbose=1
)


STEP 8 — Train Model

In [ ]:
print("Training started...")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

print("Training completed")


STEP 9 — Save Model

In [ ]:
model.save("clothe_classifier_efficientnet_lite.h5")
print("Model saved as clothe_classifier_efficientnet_lite.h5")
